In [1]:
from keys import username,password,monster_auth_token,monster_user_id
import pandas as pd
import requests
import warnings
import time
from seleniumbase import Driver
from datetime import datetime, timedelta
warnings.filterwarnings('ignore')

Functions

In [2]:

def login(username, password):

    driver = Driver(browser="chrome", headless=False)
  
    driver.open("https://www.bovada.lv/")
    driver.click('//*[@id="headerUnloggedHiddenLogin"]')
    time.sleep(2)
    driver.send_keys('//*[@id="email"]', username)
    time.sleep(2)
    driver.send_keys('//*[@id="login-password"]', password)
    time.sleep(2)
    driver.click('//*[@id="login-submit"]')
    time.sleep(60)
    cookies = driver.get_cookies()


    for i in cookies:
        if i['name'] == "_hjSessionUser_510373":
            hj_session_user = i['value']
        elif i['name'] == "_hjSession_510373":
            hj_session = i['value']
        elif i['name'] == "TS01890ddd030":
            ts = i['value']  # Assigning to `ts` as used in place_order function
        elif i['name'] == "audiences":
            audiences = i['value']  # Matching the argument name in the function call
        elif i['name'] == "sid":
            authorization = i['value']  # Matching with `authorization` in place_order
        elif i['name'] == "variant":
            variant = i['value']  # Matching the argument name
        elif i['name'] == "JSESSIONID":
            jsession_id = i['value']  # Already matched correctly
        elif i['name'] == "TS01890ddd":
            ts_ddd = i['value']  # Renaming to `ts_ddd` as used in the function call
        elif i['name'] == "wt_rla":
            wt_rla = i['value']  # Matching with `wt_rla` in place_order

    driver.quit()
    
    return hj_session_user, hj_session, ts, audiences, authorization, variant, jsession_id, ts_ddd, wt_rla


def get_balance(authorization):
    url = f'https://services.bovada.lv/services/wallet-gateway/v1/profiles/{authorization}/balances'

    # Headers for the request
    headers = {
        'authorization': f'Bearer {authorization}',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
    }

    cookies = {
    }
    response = requests.get(url, headers=headers, cookies=cookies)
    data = response.json()
    print(data[0]['amount'])  
    return float(data[0]['amount'])

def get_bets():
    url = "https://www.bovada.lv/services/sports/event/coupon/events/A/description/football?marketFilterId=def&preMatchOnly=true&eventsLimit=50&lang=en"
    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"
    }
    df = pd.DataFrame(columns=['bet_category', 'bet_category_amount', 'bet_over_under', 'game_name', 'link', 'outcome_id', 'price_id'])
    data = requests.get(url, headers=headers).json()
    
    for i in data[0]['events']: # This is all NFL games listed here
        if i['live'] == False:
            link = i['link']
            game_name = i['description']

            '''This is the link to the game'''
            url = "https://www.bovada.lv/services/sports/event/coupon/events/A/description"+link
            response = requests.get(url, headers=headers)
            data = response.json()

            for j in [0,1,2,3,4,5,6,7,8,9,10,11,12]: # For Recieving Yards, Rushing Yards, etc
                try:
                    if data[0]['events'][0]['displayGroups'][j]['description'].lower() in ["passing props","receiving props","rushing props"]:
                        for i in range(len(data[0]['events'][0]['displayGroups'][j]['markets'])):
                            over = 0
                            under = 1
                            for k in [over,under]:
                                try:
                                    outcome_id = data[0]['events'][0]['displayGroups'][j]['markets'][i]['outcomes'][k]['id']
                                    price_id = data[0]['events'][0]['displayGroups'][j]['markets'][i]['outcomes'][k]['price']['id']
                                    # Strikeouts etc
                                    bet_category = data[0]['events'][0]['displayGroups'][j]['markets'][i]['descriptionKey']
                                    # Over/Under
                                    bet_over_under = data[0]['events'][0]['displayGroups'][j]['markets'][i]['outcomes'][k]['description']
                                    # 1.5, 0.5, etc
                                    bet_category_amount = data[0]['events'][0]['displayGroups'][j]['markets'][i]['outcomes'][k]['price']['handicap']

                                    new_df = pd.DataFrame({
                                        'bet_category': [bet_category],
                                        'bet_category_amount': [bet_category_amount],
                                        'bet_over_under': [bet_over_under], 
                                        'game_name': [game_name],
                                        'link': [link],
                                        'outcome_id': [outcome_id],
                                        'price_id': [price_id]
                                    })

                                    # Use pd.concat instead of append (since append is deprecated)
                                    df = pd.concat([df, new_df], ignore_index=True)

                                except:
                                    continue
                except:
                    pass
    return df


def place_order(authorization, hj_session_user, hj_session, ts, audiences, variant, jsession_id, ts_ddd, wt_rla, outcome_id, price_id, money_per_bet):
        # URL for the betslip API
    url = "https://services.bovada.lv/services/sports/bet/betslip"

    # Headers to mimic the original request
    headers = {
        "authority": "services.bovada.lv",
        "method": "POST",
        "path": "/services/sports/bet/betslip",
        "scheme": "https",
        "accept": "application/json, text/plain, */*",
        "accept-encoding": "gzip, deflate, br, zstd",
        "accept-language": "en-US,en-GB;q=0.9,en;q=0.8",
        "authorization": f"Bearer {authorization}", 
        "content-type": "application/json",
        "origin": "https://services.bovada.lv",
        "referer": "https://services.bovada.lv/",
        "sec-ch-ua": '"Chromium";v="128", "Not;A=Brand";v="24", "Google Chrome";v="128"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"Windows"',
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "same-origin",
        "sec-fetch-site": "same-origin",
        #"traceid": "20f4aef9-c4f3-4054-97af-1a93dfffab1a", # This is a random value?? Because it doesnt come in the cookies from login
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36",
        "x-channel": "desktop",
        "x-session-id": f"Bearer {authorization}"  
    }

    # Cookies for the request
    cookies = {
        "_hjSessionUser_510373": hj_session_user,
        "_hjSession_510373": hj_session,
        "TS01890ddd030": ts,
        "audiences": audiences,
        "sid": authorization,
        "variant": variant,
        "JSESSIONID": jsession_id,
        "TS01890ddd": ts_ddd,
        "wt_rla": wt_rla
    }

    # JSON payload for the betslip
    payload = {
        "device": "DESKTOP",
        "channel": "WEB_BS",
        "selections": {
            "selection": [
                {
                    "outcomeId": str(outcome_id),
                    "id": 0,
                    "system": "A",
                    "priceId": str(price_id),
                    "oddsFormat": "AMERICAN"
                }
            ]
        },
        "groups": {
            "group": [
                {
                    "id": 0,
                    "type": "STRAIGHT",
                    "groupSelections": [
                        {
                            "groupSelection": [
                                {
                                    "selectionId": 0,
                                    "order": 0,
                                    "isBanker": False
                                }
                            ]
                        }
                    ]
                }
            ]
        },
        "bets": {
            "bet": [
                {
                    "betGroups": {
                        "groupId": [0]
                    },
                    "betType": "SINGLE",
                    "isBox": False,
                    "oddsFormat": "AMERICAN",
                    "specifyingRisk": True,
                    "stakePerLine": int(money_per_bet*100),
                }
            ]
        }
    }

    # Send the POST request
    response = requests.post(url, headers=headers, cookies=cookies, json=payload)
    print(response.status_code)


def update_monster_csv():
    url = f'https://api.monster.bet/monster/v2/player_picks?user_id={monster_user_id}&league=All&market=All'
    headers = {
        'authorization': monster_auth_token
    }
    payload = {
        'user_id': monster_user_id,
        'league': 'All',
        'market': 'All'
    }

    response = requests.post(url, headers=headers, params=payload)
    print(response.status_code)
    data  = response.json()
    df = pd.DataFrame(data)
    df_real = pd.DataFrame()

    df_real['game_name'] = df['away_team'] + ' @ ' + df['home_team']
    df_real['game_time'] = df['game_start'][:-6]
    df_real['bet_category'] = df['market']
    df_real['player_names'] = df['bet_name'].apply(lambda x: x.split(' Under')[0] if 'Under' in x else x.split(' Over')[0] if 'Over' in x else x)
    df_real['edge'] = df['edge']
    df_real['prop'] = df['simulated_prob']
    df_real['bet'] = df['bet_name']
    df_real['bet_over_under'] = ''
    df_real['bet_category_amount'] = ''


    drop_indices = []

    for i in range(len(df_real)):
        df_real['bet'][i] = df_real['bet'][i].split(' ')[-1] + ' ' + df_real['bet'][i].split(' ')[-2]
        df_real['bet_over_under'][i] = df_real['bet'][i].split(' ')[1]
        df_real['bet_category_amount'][i] = df_real['bet'][i].split(' ')[0]

    df_real = df_real.drop(columns=['bet'])

    df_real = df_real.sort_values(by='game_name')
    for i in range(len(df_real)):
        if len(str(df_real['game_time'][i])) < 5:
            game_name = df_real['game_name'][i]
            for j in range(len(df_real)):
                if j != i and df_real['game_name'][j] == game_name:
                    df_real['game_time'][i] = df_real['game_time'][j]
                    break

        eastern_time = datetime.fromisoformat(str(df_real['game_time'][i]))
        central_time = eastern_time - timedelta(hours=1)
        df_real['game_time'][i] = central_time

        '''CONDITIONS TO CHOOSE WHERE TO BET'''
        if df_real['prop'][i] >= 0.6:
            pass
        else:
            drop_indices.append(i)

    """DROPPING ROWS THAT DON'T MEET CONDITIONS"""
    df_real = df_real.drop(drop_indices)
    df_real.to_csv('monster.csv', index=False)
    print('number of bets in monster: ',len(df_real))
    return df_real


def clean_bovada_df(df):
    drop_count = 0
    
    nfl_teams_schedules = {"Arizona Cardinals": "ARI", "Atlanta Falcons": "ATL", "Baltimore Ravens": "BAL", "Buffalo Bills": "BUF", "Carolina Panthers": "CAR", "Chicago Bears": "CHI", "Cincinnati Bengals": "CIN", "Cleveland Browns": "CLE", "Dallas Cowboys": "DAL", "Denver Broncos": "DEN", "Detroit Lions": "DET", "Green Bay Packers": "GB", "Houston Texans": "HOU", "Indianapolis Colts": "IND", "Jacksonville Jaguars": "JAX", "Kansas City Chiefs": "KC", "Las Vegas Raiders": "LV", "Los Angeles Chargers": "LAC", "Los Angeles Rams": "LAR", "Miami Dolphins": "MIA", "Minnesota Vikings": "MIN", "New England Patriots": "NE", "New Orleans Saints": "NO", "New York Giants": "NYG", "New York Jets": "NYJ", "Philadelphia Eagles": "PHI", "Pittsburgh Steelers": "PIT", "San Francisco 49ers": "SF", "Seattle Seahawks": "SEA", "Tampa Bay Buccaneers": "TB", "Tennessee Titans": "TEN", "Washington Commanders": "WAS"}
    prop_types = {'Total Receiving Yards':"player_reception_yds","Total Receptions":"player_receptions","Total Rushing Yards":"player_rush_yds","Total Player Passing Yards":"player_pass_yds","Total Passing Touchdowns":"player_anytime_td"}

    df['player_names'] = '' 
    for i in range(len(df)):
        
        try:
            name_set = df['game_name'][i]
            name1 = name_set.split(' @ ')[0]
            name2 = name_set.split(' @ ')[1]
            if name1 in nfl_teams_schedules.keys():
                name1 = nfl_teams_schedules[name1]
            if name2 in nfl_teams_schedules.keys():
                name2 = nfl_teams_schedules[name2]
            name_set = name1 + ' @ ' + name2
            df['game_name'][i] = name_set.upper()


            initial_bet_category_list = df['bet_category'][i].split(' - ')

            df['player_names'][i] = initial_bet_category_list[1]
            df['bet_category'][i] = initial_bet_category_list[0].strip(" O/U")
            df['bet_category'][i] = prop_types[df['bet_category'][i]]

        except:
            df=df.drop(i)
            drop_count += 1
    
    #print('number of bets in bovada: ',len(df))
    #print('number of bets dropped: ',drop_count)
    return df

def generate_orders_df(all_bets_df_cleaned, monster_df):
    all_bets_df_cleaned = all_bets_df_cleaned.reset_index(drop=True)
    monster_df = monster_df.reset_index(drop=True)
    
    orders_df = monster_df.copy()
    orders_df['price_id'] = ''
    orders_df['outcome_id'] = ''
    for i in range(len(monster_df)):
        for j in range(len(all_bets_df_cleaned)):
            if (monster_df['game_name'][i].lower() == all_bets_df_cleaned['game_name'][j].lower()
            and monster_df['bet_category'][i].lower() == all_bets_df_cleaned['bet_category'][j].lower()
            and monster_df['player_names'][i].lower() == all_bets_df_cleaned['player_names'][j].lower()
            and monster_df['bet_over_under'][i].lower() == all_bets_df_cleaned['bet_over_under'][j].lower()
            and str(monster_df['bet_category_amount'][i]).lower() == str(all_bets_df_cleaned['bet_category_amount'][j]).lower()):

                orders_df['price_id'][i] = all_bets_df_cleaned['price_id'][j]
                orders_df['outcome_id'][i] = all_bets_df_cleaned['outcome_id'][j]

    # Only keep the rows that have a price_id
    orders_df = orders_df[orders_df['price_id'] != '']
    print('number of bets: ',len(orders_df))
    with open ('final_bets.csv','w') as f:
        orders_df.to_csv(f)
    return orders_df



Execution Code

In [5]:

wager_percent = 0.9
monster_df = update_monster_csv()
all_bets_df = get_bets()
all_bets_df_cleaned = clean_bovada_df(all_bets_df)
orders_df = generate_orders_df(all_bets_df_cleaned, monster_df).reset_index(drop=True)

if len(orders_df) > 15:
        hj_session_user, hj_session, ts, audiences, authorization, variant, jsession_id, ts_ddd, wt_rla = login(username, password)
        initial_money = float(get_balance(authorization))
        money_per_bet = float(round(float(initial_money*wager_percent)/len(orders_df),2))
        print(f'Money Per Bet: {money_per_bet}')

        with open("initial_money.txt",'a') as f:
            f.write(f'{initial_money}\n{datetime.now()}\n---------------------------------\n')

        for i in range(len(orders_df)):

            outcome_id = orders_df['outcome_id'][i]
            price_id = orders_df['price_id'][i]
            place_order(authorization, hj_session_user, hj_session, ts, audiences, variant, jsession_id, ts_ddd, wt_rla, outcome_id, price_id, money_per_bet)
            print(f'Placed order {i+1} of {len(orders_df)}')
            print(f".........Waiting to avoid suspicion........")
            time.sleep(5)
        print('All bets placed..............')

else: 
    print(f'Not enough bets to place {len(orders_df)}')



200
number of bets in monster:  109
number of bets:  29


*** chromedriver to download = 130.0.6723.91 (Latest Stable) 

https://storage.googleapis.com/chrome-for-testing-public/130.0.6723.91/win64/chromedriver-win64.zip ...
Download Complete!

Extracting ['chromedriver.exe'] from chromedriver-win64.zip ...
Unzip Complete!

The file [chromedriver.exe] was saved to:
C:\Users\abhap\AppData\Local\Programs\Python\Python312\Lib\site-packages\seleniumbase\drivers\chromedriver.exe

Making [chromedriver.exe 130.0.6723.91] executable ...
[chromedriver.exe 130.0.6723.91] is now ready for use!

18.67
Money Per Bet: 0.58
200
Placed order 1 of 29
.........Waiting to avoid suspicion........
200
Placed order 2 of 29
.........Waiting to avoid suspicion........
200
Placed order 3 of 29
.........Waiting to avoid suspicion........
200
Placed order 4 of 29
.........Waiting to avoid suspicion........
200
Placed order 5 of 29
.........Waiting to avoid suspicion........
200
Placed order 6 of 29
.........Wait